<a href="https://colab.research.google.com/github/Daramluv/Google_colab/blob/main/%22Gurobi_Based_Multi_Period_Production_Planning_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 40.7 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [3]:

# Create a new Gurobi model
model = gp.Model("MultiPeriodProductionPlanning")

# Data
noOfPeriods = 6
holdingCost =0.05
openingInventory=5000
productionCapacity = 30000
storageCapacity=10000
productionCost= np.array([12.50,12.55,12.70,12.80,12.85,12.95])
monthlyDemand=np.array([10000,15000,30000,35000,25000,10000])

Restricted license - for non-production use only - expires 2026-11-23


In [4]:
# Create the decision variables
x = {}
for j in range(noOfPeriods):
    x[j] = model.addVar(vtype=GRB.INTEGER, name=f"x_{j}")

In [5]:

# Set objective: maximize profit
model.setObjective(
    gp.quicksum((x[j] * productionCost[j] +
                 (openingInventory + gp.quicksum( x[i]  - monthlyDemand[i] for i in range(j) )) * productionCost[j] * holdingCost  for j in range(noOfPeriods) )),
    GRB.MINIMIZE,
)

In [6]:
# Constraints


# Ensure Production capacity
for j in range(noOfPeriods):
    model.addConstr(x[j] <= productionCapacity, f"Production Capacity {j}")

# Ensure demand is met
for j in range(noOfPeriods):
    model.addConstr(gp.quicksum((x[j]  - monthlyDemand[j])  for j in range(noOfPeriods))  + openingInventory >= 0, f"Closing Stock{j}")

In [7]:

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in range( noOfPeriods):
        if x[j].x > 0:
            print(f"Produce {j + 1}",x[j].x )

    print(f"Total Profit: ${model.objVal:.2f}")
else:
    print("No solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 6 columns and 42 nonzeros
Model fingerprint: 0x42b0524f
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 1e+05]
Presolve removed 12 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 1.48448e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.484475000000e+06, best bound 1.484475000000e+06, gap 0.0000%
Optimal solution found:
Produce 3 30000.0
Produce 4 30000.0
Produce 5 30000.0
Produce 6 300